# Week 3 Assignment: Part 1

### Library Import

In [1]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium
import pandas as pd
import numpy as np
import seaborn as sns
import requests
import lxml.html as lh
import bs4 as bs
import urllib.request
import matplotlib.pyplot as plt
print('Import Complete')


Solving environment: ...working... done

# All requested packages already installed.

Import Complete


In [2]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\alexa\Anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0




geopy-2.0.0          | 63 KB     |            |   0% 
geopy-2.0.0          | 63 KB     | ##5        |  25% 
geopy-2.0.0          | 63 KB     | ########## | 100% 

geographiclib-1.50   | 34 KB     |            |   0% 
geographiclib-1.50  

### Import and Scraping of Data

In [7]:

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
get = requests.get(url)
soup = bs.BeautifulSoup(get.content,'lxml')
tab = soup.find_all('table')[0]
dat = pd.read_html(str(tab))
df = pd.read_json(dat[0].to_json(orient='records'))
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Removal of 'Not Assigned' Values and Replacement of Neighborhood Values

In [8]:
val=df[df['Borough']!= 'Not Assigned']
val = val.groupby(['Borough', 'Postal Code'], as_index=False).agg(','.join)
val['Neighbourhood'] = np.where(val['Neighbourhood'] == 'Not assigned', val['Borough'], val['Neighbourhood'])
val.head()

,Borough,Postal Code,Neighbourhood
0,Central Toronto,M4N,Lawrence Park
1,Central Toronto,M4P,Davisville North
2,Central Toronto,M4R,"North Toronto West, Lawrence Park"
3,Central Toronto,M4S,Davisville
4,Central Toronto,M4T,"Moore Park, Summerhill East"


In [9]:
val.shape

(180, 3)

# Week 3 Assignment Part 2

In [10]:
ll = "http://cocl.us/Geospatial_data"
ll = pd.read_csv(ll)
ll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
lll = pd.merge(val, ll, on='Postal Code')
lll.head()

,Borough,Postal Code,Neighbourhood,Latitude,Longitude
0,Central Toronto,M4N,Lawrence Park,43.728020,-79.388790
1,Central Toronto,M4P,Davisville North,43.712751,-79.390197
2,Central Toronto,M4R,"North Toronto West, Lawrence Park",43.715383,-79.405678
3,Central Toronto,M4S,Davisville,43.704324,-79.388790
4,Central Toronto,M4T,"Moore Park, Summerhill East",43.689574,-79.383160


# Week 3 Assignment Part 3

In [13]:
lat = 43.653963
long = -79.387207
print('Lat/long of Toronto is {}, {}.'.format(lat, long))

Lat/long of Toronto is 43.653963, -79.387207.


In [27]:
Map = folium.Map(location=[lat, long], zoom_start=12)
for lat, long, borough, neighborhood in zip(lll['Latitude'], lll['Longitude'], lll['Borough'], lll['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Map) 
    
Map